In [2]:
# Import all necessary tools

import os
import numpy as np
import matplotlib.pyplot as plt

from glob import glob
from skimage.io import imread,imsave
from tqdm import tqdm
from pathlib import Path

import pickle
 
from optimal3dtracks.utils_3Dtracking import (
    calculate_Gaussian_parameters,calculate_track_sections,concatenate_track_sections,
    save_intensity_for_TrackMate,save_as_TrackMate,load_TrackMate,
    show_4d_with_contours, project_colours, generate_tree)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# Data directory and results directories

folder = 'Embryo1/'
intensity_folder = 'intensity'
label_folder = 'labels'
tracks_folder = 'tracks'
TrackMate_folder = 'TrackMate'
Gaussian_parameters_folder = 'tracks/Gaussian_parameters'
affine_folder = 'tracks/affines'
track_sections_folder = 'tracks/track_sections'

if not os.path.isdir(folder + tracks_folder):
    os.makedirs(folder + tracks_folder)
if not os.path.isdir(folder + Gaussian_parameters_folder):
    os.makedirs(folder + Gaussian_parameters_folder)
if not os.path.isdir(folder + affine_folder):
    os.makedirs(folder + affine_folder)
if not os.path.isdir(folder + track_sections_folder):
    os.makedirs(folder + track_sections_folder)
if not os.path.isdir(folder + TrackMate_folder):
    os.makedirs(folder + TrackMate_folder)


## Automated tracking using Optimal3dTracks

In [ ]:
# Calculate and save Gaussian parameters for each region at each time point

# MAKE SURE THAT label_files AND intensity_files ARE ORDERED BY TIME

label_files = sorted(glob(folder + label_folder + '/*.tif'))[:]
intensity_files = sorted(glob(folder + intensity_folder + '/*.tif'))[:]
save_folder = folder + Gaussian_parameters_folder

resolution = np.array([2,0.174,0.174]) # eg. in um

calculate_Gaussian_parameters(label_files, intensity_files, save_folder, resolution)
    

In [ ]:
# Create track segments between consecutive time points

# MAKE SURE THAT label_files, intensity_files, AND Gaussian_parameter_files ARE ORDERED BY TIME

# Load data files
label_files = sorted(glob(folder + label_folder + '/*.tif'))[:]
intensity_files = sorted(glob(folder + intensity_folder + '/*.tif'))[:]
Gaussian_parameter_files = sorted(glob(folder + Gaussian_parameters_folder + '/*'))[:]
save_folder_for_affine = folder + affine_folder
save_folder_for_tracks = folder + track_sections_folder

# Extract frame numbers from intensity_files
frames = [int(os.path.splitext(os.path.basename(file))[0][10:]) for file in intensity_files]
print(frames)

resolution = np.array([2,0.174,0.174]) # in um
max_number_of_cells_per_timepoint = 1000 # it's unlikely that any time point contains more than 1000 
# segmented cells (including dirt). Make this as large as you like, but make sure that 
# n_timepoints*max_number_of_cells_per_timepoint < 2**32 to avoid running into number representation issues

include_splits = True
include_merges = False # Allowing merges to be detected could overcomplicate tracking
        # and it's usually easier to add merges manually because they are very rare

calculate_track_sections(label_files, intensity_files, Gaussian_parameter_files, frames, save_folder_for_affine,
                              save_folder_for_tracks, resolution, max_number_of_cells_per_timepoint,
                              include_splits,include_merges)

## This might give a few "Sinkhorn did not converge" warnings, but it's fine as long as all the marginals are small enough 
# (< 0.01)

In [ ]:
# Concatenate track segments

# MAKE SURE THAT Gaussian_parameter_files AND track_files ARE ORDERED BY TIME

Gaussian_parameter_files = sorted(glob(folder + Gaussian_parameters_folder + '/*'))[:]
track_files = sorted(glob(folder + track_sections_folder + '/*'))[:]
save_folder = folder

max_number_of_cells_per_timepoint = 1000 # SAME AS ABOVE

track_df, split_df, merge_df = concatenate_track_sections(track_files,Gaussian_parameter_files,
                                                          save_folder,max_number_of_cells_per_timepoint)

## Track corrections using basic Fiji features

We implemented a function to export and import tracks to and from Fiji's (https://imagej.net/software/fiji/) TrackMate tool (https://imagej.net/plugins/trackmate/). Make sure that the intensity image saved below is in the same folder as the .xml file you're working on and is called "intensity.tif". Feel free to change the name of the .xml file and/or save as many checkpoints (in the same folder) as you wish.


In [ ]:
# Export tracks as .xml readable by Fiji's TrackMate tool for track corrections

# MAKE SURE THAT label_files AND intensity_files ARE ORDERED BY TIME

# Load segmentations and tracks
label_files = sorted(glob(folder + label_folder + '/*.tif'))[:]
intensity_files = sorted(glob(folder + intensity_folder + '/*.tif'))[:]
with open(folder + '/Tracks_full', "rb") as fp: 
    track_df, split_df, merge_df = pickle.load(fp)

dimensions = 3 # You can specify whether you'd like to correct tracks in 3D (dimensions = 3) or on the 2D MIPs (dimensions = 2)
resolution = np.array([2,0.174,0.174]) # eg. in um

frames = np.unique(track_df['timepoint'])
max_number_of_cells_per_timepoint = 1000 # SAME AS ABOVE

# Save intensity.tif for TrackMate
save_folder = folder + TrackMate_folder
save_intensity_for_TrackMate(intensity_files,dimensions,resolution,save_folder)

# Save tracks as .xml
base_file = 'base_file.xml'
save_file = folder + TrackMate_folder + '/initial_tracks.xml'

save_as_TrackMate(track_df,split_df,merge_df,label_files,dimensions,resolution,base_file,save_file)

Now, open Fiji, go to Plugins/Tracking/Load a TrackMate file, and select the .xml file you just saved. Perform track corrections as you like. Keep in mind that adding new spots and moving spots will not translate into the segmentations, so try avoiding these if you'd like to use any of the re-colouring or display features below. Deleting spots/tracks is fine. 

In [ ]:
# Save re-coloured track files (i.e. where tracks are indicated by preservation of label colour over time)

# MAKE SURE THAT label_files ARE ORDERED BY TIME

# Load label files and tracks
label_files = sorted(glob(folder + label_folder + '/*.tif'))[:]
file_path = folder + TrackMate_folder + 'final_tracks.xml' # name of your final corrected track file
track_df,split_df,merge_df = load_TrackMate(file_path)

max_number_of_cells_per_timepoint = 1000 # SAME AS ABOVE


for count,file in enumerate(label_files):
    
    print(file)
    
    frame = count+1
    print('frame = ' + str(frame))
    
    # Name the output file (this is normally part of the intensity image file name)
    output_file_name = os.path.splitext(os.path.basename(file))[0][0:-6] + '_tracks.tif' 
    
    # Load segmentation
    Y = imread(file,is_ome=False)
    
    # Re-colour regions according to track_df
    replace_dict = dict(zip(track_df['label'][track_df['timepoint']==frame] % max_number_of_cells_per_timepoint, 
                            track_df['track_id'][track_df['timepoint']==frame]))
    def replace(element):
        return replace_dict.get(element,0)
    vfunc = np.vectorize(replace)
    Y = vfunc(Y)
    
    # Save re-coloured track files
    imsave(folder + tracks_folder + '/' + output_file_name,  Y.astype('uint16'), imagej=True, 
           resolution=1/resolution[1:3],
           metadata={
               'spacing': resolution[0],
               'unit': 'um',
               'axes': 'ZYX'
           }) 
    

## Display features

In [ ]:
# A python widget for inspecting the segmented tracks as overlay on the intensity image

# MAKE SURE THAT label_files AND intensity_files ARE ORDERED BY TIME

label_files = sorted(glob(folder + tracks_folder + '/*.tif'))[:]
intensity_files = sorted(glob(folder + intensity_files + '/*.tif'))[:]

labels = np.concatenate([np.expand_dims(imread(file,is_ome=False),axis=0), for file in tqdm(label_files)])
intensities = np.concatenate([np.expand_dims(imread(file,is_ome=False),axis=0), for file in tqdm(intensity_files)])

# Show 4D images with countours
show_4d_with_contours(np.concatenate(intensities),np.concatenate(labels))

In [ ]:
# Show a top-down view of the tracks

label_image = imread(sorted(glob(folder + tracks_folder + '/*.tif'))[0],is_ome=False)

plt.imshow(project_colours(label_image))

In [ ]:
# Plot division tree

# Load tracks
track_df,split_df,merge_df = load_TrackMate(file_path)
    
tree, start, end = generate_tree(track_df, split_df, merge_df)